In [6]:
import librosa
import librosa.display
import IPython
import numpy as np
from scipy import linalg
from scipy import stats
import scipy.signal
from scipy.linalg import toeplitz
from scipy.signal import wiener
from scipy import signal
import matplotlib.pyplot as plt
import sounddevice as sd # only needed for playing
import soundfile as sf # only needed if playing does not work
from scipy.io import loadmat
from scipy import signal
from IPython.display import Audio
from scipy.fftpack import ifft, idct, dct
import math
from sklearn.linear_model import LassoLars


problem= 'problem3_1'
matdata = loadmat(f'{problem}.mat')
n = matdata['n']
x = matdata['x']
l = 2**9;
N =len(x); 
# Construct the sensing matrix
positions = np.random.choice(l, N, replace=False)
B = np.zeros(shape=(N, l))
for i in range(0, N):
    B[i, positions[i]] = 1

BF = idct(B, axis=1)  
y=B.T@x;
# Use LassoLars
model = LassoLars(alpha=0.01, fit_intercept=False, normalize=False,  max_iter=int(1e6))
model.fit(BF, y)
solsB = model.coef_
print(f'Number of non-zero weights: {np.count_nonzero(solsB)}')

ValueError: Found input variables with inconsistent numbers of samples: [32, 512]